# Setup

In [1]:
REDIS_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

REDIS_TOTAL_NODES = 6
REDIS_NODE_IPS = ["10.15.20.2"] * REDIS_TOTAL_NODES
REDIS_NODE_NAMES = [f"redis-node-{i+1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_HOSTNAMES = [
    f"{REDIS_NODE_NAMES[i]}.mavasbel.vpn.itam.mx" for i in range(REDIS_TOTAL_NODES)
]
REDIS_NODE_PORTS = [f"{6380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_BUS_PORTS = [f"{16380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]

REDIS_WORKDIR = "/data"

REDIS_ADMIN_PASSWORD = "redis"
REDIS_DEFAULT_PASSWORD = "redis"
REDIS_INIT_USER = "redis"
REDIS_INIT_PASSWORD = "redis"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Session creation

In [3]:
import pprint
from redis.cluster import RedisCluster, ClusterNode
from redis.cluster import LoadBalancingStrategy

redis_nodes = [
    ClusterNode(f"{REDIS_NODE_HOSTNAMES[i]}", REDIS_NODE_PORTS[i])
    for i in range(0, REDIS_TOTAL_NODES)
]
pprint.pprint(f"🔗 Connecting to: {redis_nodes}")

try:
    redis_cluster = RedisCluster(
        startup_nodes=redis_nodes,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,  # decode_responses=True converts bytes to strings automatically
        load_balancing_strategy=LoadBalancingStrategy.RANDOM_REPLICA,
        searedis_clusterh_all_nodes=True,
        require_full_coverage=False,
    )
    cluster_status = redis_cluster.cluster_info()["cluster_state"]
    nodes_count = len(redis_cluster.get_nodes())
    state_icon = "🟢" if cluster_status == "ok" else "🔴"
    print("✅ Cluster connected")
    print(f"{state_icon} Cluster State: {cluster_status.upper()}")
    print(f"🌐 Nodes Discovered: {nodes_count}")
except Exception as e:
    print(f"Connection failed: {e}")

('🔗 Connecting to: '
 '[[host=redis-node-1.mavasbel.vpn.itam.mx,port=6381,name=redis-node-1.mavasbel.vpn.itam.mx:6381,server_type=None,redis_connection=None], '
 '[host=redis-node-2.mavasbel.vpn.itam.mx,port=6382,name=redis-node-2.mavasbel.vpn.itam.mx:6382,server_type=None,redis_connection=None], '
 '[host=redis-node-3.mavasbel.vpn.itam.mx,port=6383,name=redis-node-3.mavasbel.vpn.itam.mx:6383,server_type=None,redis_connection=None], '
 '[host=redis-node-4.mavasbel.vpn.itam.mx,port=6384,name=redis-node-4.mavasbel.vpn.itam.mx:6384,server_type=None,redis_connection=None], '
 '[host=redis-node-5.mavasbel.vpn.itam.mx,port=6385,name=redis-node-5.mavasbel.vpn.itam.mx:6385,server_type=None,redis_connection=None], '
 '[host=redis-node-6.mavasbel.vpn.itam.mx,port=6386,name=redis-node-6.mavasbel.vpn.itam.mx:6386,server_type=None,redis_connection=None]]')
✅ Cluster connected
🟢 Cluster State: OK
🌐 Nodes Discovered: 3


In [4]:
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

In [5]:
from redis import Redis
import pandas as pd

cluster_info = []
config_get = []
for i in range(REDIS_TOTAL_NODES):
    redis = Redis(
        host=REDIS_NODE_HOSTNAMES[i],
        port=REDIS_NODE_PORTS[i],
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    cluster_info.append(redis.execute_command("CLUSTER INFO"))
    config_get.append(redis.config_get())

display(pd.DataFrame(cluster_info).transpose())
display(pd.DataFrame(config_get).transpose())

,0,1,2,3,4,5
cluster_state,ok,ok,ok,ok,ok,ok
cluster_slots_assigned,16384,16384,16384,16384,16384,16384
cluster_slots_ok,16384,16384,16384,16384,16384,16384
cluster_slots_pfail,0,0,0,0,0,0
cluster_slots_fail,0,0,0,0,0,0
cluster_known_nodes,6,6,6,6,6,6
cluster_size,3,3,3,3,3,3
cluster_current_epoch,7,7,7,7,7,7
cluster_my_epoch,1,7,3,3,1,7
cluster_stats_messages_ping_sent,55446,2242,56022,55689,56132,56163


,0,1,2,3,4,5
cluster-slot-migration-write-pause-timeout,10000,10000,10000,10000,10000,10000
cluster-announce-bus-port,16381,16382,16383,16384,16385,16386
hz,10,10,10,10,10,10
tls-client-key-file,,,,,,
shutdown-on-sigint,default,default,default,default,default,default
search-union-iterator-heap,20,20,20,20,20,20
replica-full-sync-buffer-limit,0,0,0,0,0,0
socket-mark-id,0,0,0,0,0,0
repl-diskless-load,disabled,disabled,disabled,disabled,disabled,disabled
active-defrag-threshold-upper,100,100,100,100,100,100


In [6]:
import pprint
import pandas as pd
from IPython.display import display

try:
    print("🛰️ Cluster Nodes View")
    display(
        pd.DataFrame(redis_cluster.execute_command("CLUSTER NODES"))
        .transpose()
        .sort_index()
        .transpose()
        .sort_index()
    )
except Exception as e:
    pprint.pprint(f"Error checking survivors: {e}")

🛰️ Cluster Nodes View


,10.15.20.2:6381,10.15.20.2:6382,10.15.20.2:6383,10.15.20.2:6384,10.15.20.2:6385,10.15.20.2:6386
connected,True,True,True,True,True,True
epoch,1,7,3,3,1,7
flags,"myself,master",slave,master,slave,slave,master
hostname,,,,,,
last_ping_sent,0,0,0,0,0,0
last_pong_rcvd,0,1768184626528,1768184626527,1768184626528,1768184627030,1768184626022
master_id,-,e91f1450397a84691fe503e89610594204c43301,-,d905699d60e203546755a4701c3cac6944262566,dff1ea7734855a870bc6d37f9d1777e292cdebc0,-
migrations,[],[],[],[],[],[]
node_id,dff1ea7734855a870bc6d37f9d1777e292cdebc0,e5bd9193db4bc7bc341229e36b57bf2df74103fe,d905699d60e203546755a4701c3cac6944262566,939a4b34e15987d7eee6e8a4e38d87a9a22df900,1d6057a1b6a7a6a6cb7e09074301f792de533ec3,e91f1450397a84691fe503e89610594204c43301
slots,"[[0, 5460]]",[],"[[10923, 16383]]",[],[],"[[5461, 10922]]"


In [7]:
import uuid
from typing import cast
from redis import Redis

# 1. Reset statistics on all nodes
print("Emptying stats on all nodes...")
# for node in redis_cluster.get_nodes():
for node in redis_nodes:
    redis = Redis(
        host=node.host,
        port=node.port,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    redis.config_resetstat()

# 2. Perform a burst of reads on multiple keys
print("Performing 1000 reads across 10 keys...")
test_keys = [f"{uuid.uuid4()}" for i in range(10)]
for k in test_keys:
    redis_cluster.set(k, str(uuid.uuid4()))  # Ensure keys exist
for i in range(1000):
    redis_cluster.get(test_keys[i % REDIS_TOTAL_NODES])

# 3. Check who actually processed the 'GET' command
print("\n📊 ACTUAL Execution Stats (From Redis Engines)")
for node in redis_cluster.get_nodes():
    node = cast(ClusterNode, node)
    redis = Redis(
        host=node.host,
        port=node.port,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    role = redis.info("replication")["role"]
    stats = redis.info("commandstats")
    get_calls = stats.get("cmdstat_get", {}).get("calls", 0)

    if get_calls > 0:
        print(f"✅ Node {node.port} ({role.upper()}): {get_calls} GETs handled")
    else:
        print(f"❌ Node {node.port} ({role.upper()}): 0 GETs handled")

Emptying stats on all nodes...
Performing 1000 reads across 10 keys...

📊 ACTUAL Execution Stats (From Redis Engines)
✅ Node 6381 (MASTER): 499 GETs handled
✅ Node 6386 (MASTER): 334 GETs handled
✅ Node 6383 (MASTER): 167 GETs handled


In [8]:
# Flushes all in every primary node
for node in redis_cluster.get_primaries():
    node = cast(ClusterNode, node)
    redis = Redis(
        host=node.host,
        port=node.port,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    redis.flushall()

In [9]:
# import time
# from redis import Redis

# idx = 4

# master_node = Redis(
#     host=REDIS_NODE_HOSTNAMES[idx],
#     port=REDIS_NODE_PORTS[idx],
#     username=REDIS_INIT_USER,
#     password=REDIS_INIT_PASSWORD,
#     decode_responses=True,
# )

# print("👀 Monitoring Cluster for Failover Events... (Kill your master now)")
# while True:
#     nodes = master_node.execute_command("CLUSTER NODES")
#     # Look for the 'fail' flag or 'failover' state
#     for line in str(nodes).split("\n"):
#         if "fail" in line or "handshake" in line:
#             print(f"⏰ {time.strftime('%H:%M:%S')} | Event Detected: {line}...")
#             break
#     time.sleep(2)

In [10]:
# from redis import Redis

# idx = 4

# replica = Redis(
#     host=REDIS_NODE_HOSTNAMES[idx],
#     port=REDIS_NODE_PORTS[idx],
#     username=REDIS_INIT_USER,
#     password=REDIS_INIT_PASSWORD,
#     decode_responses=True,
# )

# try:
#     print(
#         f"🚀 Attempting foredis_clustered takeover on port {REDIS_NODE_PORTS[idx]}..."
#     )
#     # 'TAKEOVER' is the aggressive version of failover
#     replica.execute_command("CLUSTER FAILOVER TAKEOVER")
#     print("✅ Success! The replica has promoted itself to MASTER.")
#     redis_cluster.nodes_manager.initialize()
#     print("✅ Success! The cluster has been initialized.")
# except Exception as e:
#     print(f"❌ Manual takeover failed: {e}")
#     print("Check if the Replica can actually 'see' the other Masters.")